# Get RT plan

## This code create a pthon dictionnary object with useful data of RT plan dicom.

    ### It works for static and dynamic beams with and without MLC
    ### !!! It assumes no Jaws position variation between each Control Point !!!

    ### It creates a "Rtplan.txt" summary of data (Patient Name, Patient ID, Numbers of Beam and  MLC position, Gantry angle, Cumulative Dose Coefficient t each Control Point index)

    ### It creates one "ClinigConfig-x.tb" for each beam(x) of the the RT plan this file can be used as entry for gate simulation

    ### It creates a "dynamicMLC-x.mlc" or a "staticMLC-x.mlc" file if MLC is used for each beam of the RT plan. This MLC file can used as an entry to makeMLC_dynamique.c to set MLC position at each control point index or run

    ### It creates  a time slice file for gate simulation
    
    ### ************** HELP for DICOM RT PLAN STRUCTURE ************ --> See mozilla -> bibliothèque -> pocket -> Dicom Standard Browser

In [1]:
import dicom
import matplotlib.pyplot as plt


In [2]:
#Read a dicom RTplan file ".dcm"
plan = dicom.read_file("RA1.dcm")
#plan = dicom.read_file("RTplanLung.dcm")
#plan = dicom.read_file("RTplanJeremy/static10x10mlc.dcm")
#plan = dicom.read_file("onlymlc5x5.dcm")
#plan = dicom.read_file("RTplanStatic10x10.dcm")
#print plan
print plan.BeamSequence[1].ControlPointSequence[0].IsocenterPosition

['-40', '23.0379731281013', '10']


## CREATE A PYTHON DICTIONARY OBJECT WITH USEFUL DATA OF RT PLAN

In [4]:
#Dictonary which will contains all useful data for RTpla
RTplan_file = {} 
#Create a dictionnary for each beam in RT plan dictionnary
RTplan_file['Beam'] = {}

#Get patient Name and ID
RTplan_file['Name'] = plan.PatientName
RTplan_file['ID'] = plan.PatientID


list_beam=[]                                                                               #declaration of template list for each beam

for j in range(len(plan.BeamSequence)):                                                    #Loop for each beam (arc) or static

    if plan.BeamSequence[j].TreatmentDeliveryType == 'TREATMENT':                          #Test if the beam is a treatment beam


            list_controlPoint = []                                                         #template list for number of Control points
            list_MLC=[]                                                                    #template list for MLC data
            list_MLC1=[]                                                                   #template list for MLC data
            list_JAW_X_1 = []                                                              #template list for JAW data
            list_JAW_Y_1 = []                                                              #template list for JAW data
            list_X_JAW = []                                                                #template list for JAW data
            list_Y_JAW = []                                                                #template list for JAW data
            list_CumulativeDoseReferenceCoefficient = []                                   #template list for Cumulative Dose coefficient
            list_GantryAngle = []                                                          #template list fir Gantry angle value (at each control point index)

            RTplan_file['Beam']['Beam ' + str(j)] = {}                                     #Create a dictonnary for each (treatment Beam)
            RTplan_file['Beam']['Beam ' + str(j)]['JAWS'] = {}                               #Create an index 'JAW' in Beam Dictionnary


            for i in range(len(plan.BeamSequence[j].ControlPointSequence)):                 #loop for each Control Point

                list_controlPoint.append(i)                                                 #Create a list with the number of each Control Point



                if  plan.BeamSequence[j].BeamType == 'DYNAMIC':                             #Test for DYNAMIC beam ("arc therapy")

        # MLC #                                                                             #Create a list of string with the MLCX position of each Leaf
                    list_MLC.append((str(plan.BeamSequence[j].ControlPointSequence[i].BeamLimitingDevicePositionSequence[len(plan.BeamSequence[j].ControlPointSequence[i].BeamLimitingDevicePositionSequence)-1])).replace('\'','').replace('(300a, 00b8) RT Beam Limiting Device Type        CS: MLCX','').replace('(300a, 011c) Leaf/Jaw Positions                  DS: [','').replace(']',''))
                    list_MLC1.append(map(float, list_MLC[i].split(',')))                    #Convert the list of string of MLCX position into list of float
        # JAW #
                                                                                            #Create a list of string for  X and Y Jaw position for each control point index if it exists (if JAW move for the arc therapy not used at IUCT)
                    if len(plan.BeamSequence[j].ControlPointSequence[i].BeamLimitingDevicePositionSequence) > 1:

                        list_JAW_X_1.append(str(plan.BeamSequence[j].ControlPointSequence[i].BeamLimitingDevicePositionSequence[0]).replace('\'','').replace('(300a, 00b8) RT Beam Limiting Device Type        ', '').replace('(300a, 011c) Leaf/Jaw Positions                  DS: [','').replace(']','').replace('CS: ASYMX','').replace('CS: X',''))
                        list_JAW_Y_1.append(str(plan.BeamSequence[j].ControlPointSequence[i].BeamLimitingDevicePositionSequence[1]).replace('\'','').replace('(300a, 00b8) RT Beam Limiting Device Type        ', '').replace('(300a, 011c) Leaf/Jaw Positions                  DS: [','').replace(']','').replace('CS: ASYMY','').replace('CS: Y',''))
                                                                                            #Convert the string list of X and Y Jaw position into list of float
                        list_X_JAW.append(map(float, list_JAW_X_1[i].split(',')))
                        list_Y_JAW.append(map(float, list_JAW_Y_1[i].split(',')))
        # Dose Coefficient #                                                                #Create a list of Cumulative Dose Coefficient (one per Control point)(convert str -> float)
                    list_CumulativeDoseReferenceCoefficient.append(float(str(plan.BeamSequence[j].ControlPointSequence[i].ReferencedDoseReferenceSequence[0]).replace('\'', '').replace('(300c, 0051) Referenced Dose Reference Number    IS: 1]', '').replace('[(300a, 010c) Cumulative Dose Reference Coefficie DS: ', '').replace('(300c, 0051) Referenced Dose Reference Number    IS: 1', '').replace('(300a, 010c) Cumulative Dose Reference Coefficie DS:', '')))
        # Gantry Angle #                                                                    #Create a list of Gantry Angle value (one value per Control Point)
                    list_GantryAngle.append(float(str(plan.BeamSequence[j].ControlPointSequence[i].GantryAngle)))


                                                                                        #Test for STATIC beam
            if plan.BeamSequence[j].BeamType == 'STATIC':

    # JAW #                                                                             #Create a list of string for  X and Y Jaw position for each control point index if it exists (if JAW move for the arc therapy not used at IUCT)
                list_JAW_X_1.append(str(plan.BeamSequence[j].ControlPointSequence[0].BeamLimitingDevicePositionSequence[0]).replace('\'', '').replace('(300a, 00b8) RT Beam Limiting Device Type        ', '').replace('(300a, 011c) Leaf/Jaw Positions                  DS: [', '').replace(']', '').replace('CS: ASYMX', '').replace('CS: X', ''))
                list_JAW_Y_1.append(str(plan.BeamSequence[j].ControlPointSequence[0].BeamLimitingDevicePositionSequence[1]).replace('\'', '').replace('(300a, 00b8) RT Beam Limiting Device Type        ', '').replace('(300a, 011c) Leaf/Jaw Positions                  DS: [', '').replace(']', '').replace('CS: ASYMY', '').replace('CS: Y', ''))
                                                                                        # Convert the string list of X and Y Jaw position into list of float
                list_X_JAW.append(map(float, list_JAW_X_1[0].split(',')))
                list_Y_JAW.append(map(float, list_JAW_Y_1[0].split(',')))
    # MLC #
                if len(plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDevicePositionSequence) == 3:                                              #Test if there is an MLC Sequence
                    list_MLC.append((str(plan.BeamSequence[j].ControlPointSequence[0].BeamLimitingDevicePositionSequence[len(plan.BeamSequence[j].ControlPointSequence[0].BeamLimitingDevicePositionSequence) - 1])).replace('\'','').replace('(300a, 00b8) RT Beam Limiting Device Type        CS: MLCX', '').replace('(300a, 011c) Leaf/Jaw Positions                  DS: [', '').replace(']', ''))
                    list_MLC1.append(map(float, list_MLC[0].split(',')))                    # Convert the list of string of MLCX position into list of float



    # Gantry Angle                                                                      #Create a list of Gantry Angle value (one value per Control Point)
                GantryAngle = (float(str(plan.BeamSequence[j].ControlPointSequence[0].GantryAngle)))


                                                                                            #Build the RTplan_file ditionnary with data for each control Point
            if plan.BeamSequence[j].BeamType == 'DYNAMIC':                                  #For Arc Therapy

                RTplan_file['Beam']['Beam ' + str(j)]['MLCX'] = list_MLC1
                RTplan_file['Beam']['Beam '+str(j)]['ControlPointIndex'] = list_controlPoint
                RTplan_file['Beam']['Beam ' + str(j)]['Cumulative Dose Coefficient'] = list_CumulativeDoseReferenceCoefficient
                RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'] = list_X_JAW
                RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'] = list_Y_JAW
                RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle'] = list_GantryAngle


            if plan.BeamSequence[j].BeamType == 'STATIC':                                   #For Static Beam

                RTplan_file['Beam']['Beam ' + str(j)]['ControlPointIndex'] = list_controlPoint
                RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'] = list_X_JAW
                RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'] = list_Y_JAW
                RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle'] = GantryAngle

                if len(plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDevicePositionSequence) == 3:   #if there is an MLC for this sttic Beam

                    RTplan_file['Beam']['Beam ' + str(j)]['MLCX'] = list_MLC1
print RTplan_file

print("")
print("")
print len(RTplan_file['Beam']['Beam 3']['MLCX'][0])
print len(RTplan_file['Beam']['Beam 3']['MLCX'][177])

{'Beam': {'Beam 3': {'JAWS': {'Y JAW': [[-70.0, 69.0]], 'X JAW': [[-91.0, 80.0]]}, 'MLCX': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -75.4, 24.4, -7.4, -12.4, -16.1, -18.6, -32.5, -32.4, -34.9, -38.6, -39.9, -39.9, -39.9, -42.4, -54.9, -58.6, -59.9, -59.9, -6.1, 6.6, -84.9, -86.1, -87.4, -87.4, -87.4, -89.9, -51.9, -51.0, -89.9, -63.6, -61.1, -37.4, -87.4, -54.8, -24.8, -74.9, -66.1, -23.6, -38.6, -37.5, -22.5, 59.7, 59.7, 59.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -74.8, 25.0, 22.4, 36.2, 41.2, -10.1, 48.7, -25.1, -26.7, 9.7, 51.2, 2.7, 13.4, 4.9, 7.4, 28.8, 47.4, 47.4, 47.4, 44.9, -53.8, -57.4, -46.7, -42.8, -38.8, -47.5, -30.1, -28.5, -27.6, -39.3, -17.6, -1.5, 27.4, 26.2, 24.9, 24.9, -6.3, -11.3, -15.1, -36.9, -21.9, 60.3, 60.3, 60.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -75.4, 24.4, -7.4, -12.4, -14.9, -18.6, -32.5, -32.4, -34.9, -37.4, -39.9, -39.9, -39.9, -42.4, -54.9, -57.4, -59.9, -59.9, -9

## Write RT plan Summary ".txt"

In [5]:
rt_plan_sumup=open("RTplan.txt", 'w')                                                          #Open RTplan output '.txt' file

RTplan_out.write("PATIENT NAME : " + str(RTplan_file['Name']) + '\n')                       #Write  patient name
RTplan_out.write("PATIENT ID : " + str(RTplan_file['ID']) + '\n' + '\n')                    #write patient ID


for j in range(len(plan.BeamSequence)):                                                     #loop for each beam


    if plan.BeamSequence[j].BeamType == 'DYNAMIC' and plan.BeamSequence[j].TreatmentDeliveryType == 'TREATMENT':                            #Write for Arc Therapy

        RTplan_out.write("=====================================" + '\n')
        RTplan_out.write("BEAM " + str(j) + '\n')                                                                                           #Beam NUmber

        for i in range(len(RTplan_file['Beam']['Beam ' + str(j)]['ControlPointIndex'])):                                                                 #For each Point Control

            RTplan_out.write("--------------------------------" + '\n')
            RTplan_out.write('Control Point Index : ' + str(RTplan_file['Beam']['Beam ' + str(j)]['ControlPointIndex'][i]) + '\n')           #Write Control Point Index Number
            RTplan_out.write("--------------------------------" + '\n')

            if len(plan.BeamSequence[j].ControlPointSequence[i].BeamLimitingDevicePositionSequence) > 1:                                    #If JAW position change in the arc therapy

                RTplan_out.write("X JAW :" + '\n')
                RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][i]).replace('[','').replace(']', '') + '\n')     #Write X Jaw position
                RTplan_out.write("Y JAW :" + '\n')
                RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][i]).replace('[', '').replace(']', '') + '\n')    #Write Y Jaw position
            else:                                                                                                                           #IF JAW does not move in the arc

                RTplan_out.write("X JAW :" + '\n')
                RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0]).replace('[', '').replace(']', '') + '\n')    #Wrtie initial X JAW position AT each point Control
                RTplan_out.write("Y JAW :" + '\n')
                RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][0]).replace('[', '').replace(']', '') + '\n')    #Write initial Y Jaw position AT each point control

            RTplan_out.write("MLCX :" +  '\n')
            RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][i]).replace('[','').replace(']','') + '\n')                   #write MLCX position
            RTplan_out.write("Dose reference Coefficient :" + '\n')
            RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['Cumulative Dose Coefficient'][i])+ '\n')                            #Write Cumulative dose coefficient
            RTplan_out.write("Gantry Angle :" + '\n')
            RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle'][i]) + '\n')                                          #Write Gantry angle


    if plan.BeamSequence[j].BeamType == 'STATIC' and plan.BeamSequence[j].TreatmentDeliveryType == 'TREATMENT':                             #FOR static beam


        RTplan_out.write("=====================================" + '\n')
        RTplan_out.write("BEAM " + str(j) + '\n')                                                                                           #Write Beam number
        RTplan_out.write("=====================================" +  '\n' +  '\n')


        RTplan_out.write("--------------------------------" + '\n')
        RTplan_out.write('Control Point Index : ' + str(RTplan_file['Beam']['Beam ' + str(j)]['ControlPointIndex'][0]) + '\n')               #Write Control Point index
        RTplan_out.write("--------------------------------" + '\n')
        RTplan_out.write("X JAW :" + '\n')
        RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0]).replace('[','').replace(']','') + '\n')            #Write X Jaw initial position
        RTplan_out.write("Y JAW :" + '\n')
        RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][0]).replace('[', '').replace(']','') + '\n')           #Write Y JAw position
        RTplan_out.write("Gantry Angle :" + '\n')                                                                                          #Write Gantry Angle
        RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle']) + '\n')

        if len(plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDevicePositionSequence) == 3:
            RTplan_out.write("MLCX :" + '\n')
            RTplan_out.write(str(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][0]).replace('[', '').replace(']', '') + '\n')  # write MLCX position






NameError: name 'RTplan_out' is not defined

## Create ClinicConfig.tb file with value from RT plan
    ##One ClinicalConfig.tb per beam

In [6]:
for j in range(len(plan.BeamSequence)):
    if plan.BeamSequence[j].TreatmentDeliveryType == 'TREATMENT': 
        
        with open('Config1.tb', 'r') as file:                                  #laod Config.tb   # Write JAW position in Config.tb file # At the moment only lfor the first Beam
            lines = file.readlines()

            for i in range(len(lines)):     #For Static and dynamic  write Jaw position at the first Contorl point

                    if lines[i] == "=X1=\n":

                        if RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0][0] < 1:        #If the Value is negative change it into positive

                            X1value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0][0] * -1

                        else:

                            X1value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0][0]

                        lines[i+1] = str(X1value) + '\n'

                    if lines[i] == "=X2=\n":

                        if RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0][1] < 1:

                            X2value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0][1] * -1

                        else:

                            X2value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0][1]

                        lines[i+1] = str(X2value) + '\n'

                    if lines[i] == "=Y1=\n":

                        if RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][0][0] < 1:

                            Y1value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][0][0] * -1

                        else:

                            Y1value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['X JAW'][0][0]

                        lines[i+1] = str(Y1value) + '\n'

                    if lines[i] == "=Y2=\n":

                        if RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][0][1] < 1:

                            Y2value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][0][1] * -1

                        else:

                            Y2value = RTplan_file['Beam']['Beam ' + str(j)]['JAWS']['Y JAW'][0][1]

                        lines[i+1] = str(Y2value) + '\n'

                    if lines[i] == '=MLC=\n':

                        if plan.BeamSequence[j].BeamType == 'STATIC':

                            lines[i+1] = '0' + '\n'

                            if len(plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDevicePositionSequence) == 3:

                                lines[i + 1] = '1' + '\n'

                        if plan.BeamSequence[j].BeamType == 'DYNAMIC':

                            lines[i + 1] = '2' + '\n'

                    if lines[i] == '=MLC_FILE=\n':

                        if plan.BeamSequence[j].BeamType == 'STATIC':

                            lines[i + 1] = 'staticMLC-' +str(j) + '.mlc' + '\n'

                        if plan.BeamSequence[j].BeamType == 'DYNAMIC':

                            lines[i+1] = 'dynamicMLC-' +str(j) + '.mlc' + '\n'

                    if plan.BeamSequence[j].BeamType == 'STATIC':

                        if lines[i] == '=GANTRY_STOP=\n':

                            lines[i+1] = 'X' + '\n'

                        if lines[i] == '=GANTRY=\n':                                                            #TO find for collimator angle

                            lines[i+1] = str(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle']) + '\n'

                    if plan.BeamSequence[j].BeamType == 'DYNAMIC':

                        if lines[i] == '=GANTRY=\n':                                                            #TO find for collimator angle

                            lines[i+1] = str(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle'][0]) + '\n'

                        if lines[i] == '=GANTRY_STOP=\n':

                            lines[i + 1] = str(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle'][len(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle'])-1]) + '\n'

        with open('ClinicConfig_'+str(j)+'.tb', 'w') as file:                                                                                                      #Write Config.tb with changes

            file.writelines(lines)
        print ('ClinicConfig_'+str(j)+'.tb')

ClinicConfig_2.tb
ClinicConfig_3.tb


##  Create MLC file ".mlc" 
    ###One dynamicMLC.mlc or staticMLC.mlc per beam

In [7]:
    for j in range(len(plan.BeamSequence)):
        if plan.BeamSequence[j].TreatmentDeliveryType == 'TREATMENT':
            if plan.BeamSequence[j].BeamType == 'DYNAMIC' or (plan.BeamSequence[j].BeamType == 'STATIC' and len(plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDevicePositionSequence) == 3) :

                if plan.BeamSequence[j].BeamType == 'DYNAMIC':
                    MLC_file = open('dynamicMLC-' +str(j) + '.mlc', 'w')

                if plan.BeamSequence[j].BeamType == 'STATIC':
                    MLC_file = open('staticMLC-' +str(j) + '.mlc', 'w')

                MLC_file.write('File Rev = H ' + '\n')
                MLC_file.write('Treatment = ' + plan.BeamSequence[j].BeamType + '\n')
                MLC_file.write('Last Name = ' + (RTplan_file['Name']) + '\n')
                MLC_file.write('First Name = ' + '\n')
                MLC_file.write('Patient ID = ' + (RTplan_file['ID']) + '\n')
                MLC_file.write('Number of Fields = ' + str(len(RTplan_file['Beam']['Beam '+ str(j)]['ControlPointIndex'])) + '\n')
                MLC_file.write('Model = Varian HD120' + '\n')
                MLC_file.write('Tolerance = 0.00' + '\n')
                MLC_file.write('\n')

                for i in range(len(RTplan_file['Beam']['Beam ' + str(j)]['ControlPointIndex'])):
                    #print j

                    MLC_file.write('Field = ' + 'Beam ' + str(j)+'.'+str(i) + '\n')

                    if plan.BeamSequence[j].BeamType == 'DYNAMIC':

                        MLC_file.write('Index = ' + str(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle'][i]) + '\n')

                    if plan.BeamSequence[j].BeamType == 'STATIC':

                        MLC_file.write('Index = ' + str(RTplan_file['Beam']['Beam ' + str(j)]['Gantry Angle']) + '\n')

                    MLC_file.write('Carriage Group = 1 ' + '\n')
                    MLC_file.write('Operator = ' + '\n')
                    MLC_file.write('Collimator = 0.00' + '\n')

                    if plan.BeamSequence[j].BeamType == 'DYNAMIC':

                        for ii in range(len(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][i])):

                            if ii < 60:
                                
                                MLC_file.write('Leaf ' + str(ii + 1) + 'A = ' + str(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][i][ii]) + '\n')

                            else:
                                #print j
                                #print i
                                #print ii
                                MLC_file.write('Leaf ' + str(ii - 59) + 'B = ' + str(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][i][ii]) + '\n')
                                

                        MLC_file.write('Note = 0' + '\n')
                        MLC_file.write('Shape = 0' + '\n')
                        MLC_file.write('Magnification  = 1.00' + '\n' + '\n')
                        
                print ('dynamiqueMLC-' +str(j) +'.mlc'+ ' is created')


                if plan.BeamSequence[j].BeamType == 'STATIC':

                    for ii in range(len(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][0])):

                        if ii < 60:

                            MLC_file.write('Leaf ' + str(ii + 1) + 'A = ' + str(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][0][ii]) + '\n')

                        else:

                            MLC_file.write('Leaf ' + str(ii - 59) + 'B = ' + str(RTplan_file['Beam']['Beam ' + str(j)]['MLCX'][0][ii]) + '\n')


                    MLC_file.write('Note = 0' + '\n')
                    MLC_file.write('Shape = 0' + '\n')
                    MLC_file.write('Magnification  = 1.00' + '\n' + '\n')

                    print ('staticMLC-' +str(j) + '.mlc' + ' is created')
    for i in range(0, 119):
        print (str(i) + ' --> ' + str(RTplan_file['Beam']['Beam 3']['MLCX'][177][i]))

dynamiqueMLC-2.mlc is created
dynamiqueMLC-3.mlc is created
0 --> 0.0
1 --> 0.0
2 --> 0.0
3 --> 0.0
4 --> 0.0
5 --> 0.0
6 --> 0.0
7 --> 0.0
8 --> -69.2
9 --> 6.8
10 --> -7.4
11 --> -12.4
12 --> -16.1
13 --> -16.1
14 --> -32.4
15 --> -33.6
16 --> -11.1
17 --> 28.9
18 --> -38.6
19 --> -39.9
20 --> -41.1
21 --> -39.9
22 --> -52.4
23 --> -54.9
24 --> -57.4
25 --> -57.4
26 --> -57.4
27 --> -59.9
28 --> -81.1
29 --> -84.9
30 --> -84.9
31 --> -87.4
32 --> -46.1
33 --> -43.6
34 --> 17.7
35 --> 25.1
36 --> 4.4
37 --> -4.6
38 --> -2.0
39 --> -6.3
40 --> -84.9
41 --> -82.4
42 --> -76.9
43 --> -22.4
44 --> -64.9
45 --> -58.6
46 --> -37.4
47 --> -15.7
48 --> -22.5
49 --> 59.7
50 --> 59.7
51 --> 59.7
52 --> 0.0
53 --> 0.0
54 --> 0.0
55 --> 0.0
56 --> 0.0
57 --> 0.0
58 --> 0.0
59 --> 0.0
60 --> 0.0
61 --> 0.0
62 --> 0.0
63 --> 0.0
64 --> 0.0
65 --> 0.0
66 --> 0.0
67 --> 0.0
68 --> -68.6
69 --> 7.4
70 --> 22.4
71 --> 31.2
72 --> 39.9
73 --> 43.7
74 --> 49.9
75 --> 52.4
76 --> 51.2
77 --> 51.2
78 --> -

## Plot data

In [1]:

if plan.BeamSequence[j].BeamType == 'DYNAMIC' and plan.BeamSequence[j].TreatmentDeliveryType == 'TREATMENT':

    #Calcul on Cumulative Dose coefficient
    DoseCoeff= RTplan_file['Beam']['Beam 3']['Cumulative Dose Coefficient']

    DoseCoeff2=[]
    for i in range(len(DoseCoeff)):
        DoseCoeff2.append((DoseCoeff[i]-DoseCoeff[i-1])*100)

    # PLOT the different data in function of time

    x = RTplan_file['Beam']['Beam 2']['ControlPointIndex']
    y1=RTplan_file['Beam']['Beam 2']['Gantry Angle']
    y2=RTplan_file['Beam']['Beam 2']['Cumulative Dose Coefficient']

    plt.subplot(3, 1, 1)
    plt.plot(x, y1, '-')
    plt.ylabel('Gantry angle')
    plt.xlabel('n Control Point Index')
    plt.title('Variation of Gantry angle and Cumulative Dose coefficient')

    plt.subplot(3,1,2)
    plt.plot(x, y2, '-')
    plt.ylabel('Cumulative Dose coefficient')
    plt.xlabel('n Control Point Index')


    plt.subplot(3,1,3)
    plt.plot(x, DoseCoeff2, '-')
    plt.ylabel(' Dose coeff')
    plt.xlabel('n Control Point Index')

    plt.show()

NameError: name 'plan' is not defined